# 日本蜡烛图
## pinbar 针线，锤头线
### 看涨
![](https://cdn.discordapp.com/attachments/876453115371614238/1089296024994062408/image.png)
### 看跌
![](https://cdn.discordapp.com/attachments/876453115371614238/1089295486135054356/image.png)

In [ ]:
# 抓取数据
import warnings
warnings.filterwarnings('ignore')
data = collect_data(timeframe='1d', limit=500)
df = data[data['Symbol'] == "BTC/USDT"]

In [ ]:
df.shape

In [ ]:
range(2, df.shape[0])

In [ ]:
for i in range(2, df.shape[0]):
    current = df.iloc[i, :]
    prev = df.iloc[i-1, :]
    prev_2 = df.iloc[i-2, :]
    realbody = abs(current['Open'] - current['Close'])
    candle_range = current['High'] - current['Low']
    idx = df.index[i]
    # Bullish pinbar
    df.loc[idx, 'Bullish pinbar'] = realbody <= candle_range/3 and  min(current['Open'], current['Close']) > (current['High'] + current['Low'])/2 and current['Low'] < prev['Low']
    # Bearish pinbar
    df.loc[idx, 'Bearish pinbar'] = realbody <= candle_range/3 and max(current['Open'], current['Close']) < (current['High'] + current['Low'])/2 and current['High'] > prev['High']

In [ ]:
df[df['Bullish pinbar'] == True]

# 日本蜡烛图
## 吞没形态
### 看涨
![](https://cdn.discordapp.com/attachments/876453115371614238/1089295548084920522/image.png)
### 看跌
![](https://cdn.discordapp.com/attachments/876453115371614238/1089295612589133907/image.png)

In [ ]:
for i in range(2, df.shape[0]):
    current = df.iloc[i, :]
    prev = df.iloc[i-1, :]
    prev_2 = df.iloc[i-2, :]
    realbody = abs(current['Open'] - current['Close'])
    candle_range = current['High'] - current['Low']
    idx = df.index[i]
    
    df.loc[idx,'Bullish engulfing'] = current['High'] > prev['High'] and current['Low'] < prev['Low'] and realbody >= 0.8 * candle_range and current['Close'] > current['Open']
    df.loc[idx,'Bearish engulfing'] = current['High'] > prev['High'] and current['Low'] < prev['Low'] and realbody >= 0.8 * candle_range and current['Close'] < current['Open']

In [ ]:
df[df['Bullish engulfing'] == True]

# 和谐交易 / 谐波形态
![](https://cdn.discordapp.com/attachments/876453115371614238/1089294923481747606/image.png)

In [ ]:
import numpy as np
import pandas as pd
import ccxt
import time
import dateutil
from datetime import datetime
from functools import reduce
from scipy.signal import argrelextrema
from ta import add_all_ta_features
import ta

In [ ]:
data = collect_data(timeframe='1h', limit=1000)
data['Datetime'] = pd.to_datetime(data['Datetime'])

In [ ]:
df = data[data['Symbol'] == 'BTC/USDT'].tail(100)
dt = df.Datetime

df = df.set_index('Datetime')

price = df.Close

high = df.High

low = df.Low

max_idx = list(argrelextrema(price.values, np.greater, order = 5)[0])
min_idx = list(argrelextrema(price.values, np.less, order = 5)[0])

peak_1 = price.values[max_idx]
peak_2 = price.values[min_idx]

peaks_p = list(peak_1) + list(peak_2)

peaks_idx = list(max_idx) + list(min_idx)

peaks_idx_dt = np.array(dt.values[peaks_idx])

peaks_p = np.array(list(peak_1) + list(peak_2))

plt.plot(price, c='g')
plt.xticks(rotation=45)
plt.scatter(dt.values[peaks_idx], peaks_p, c='green')

plt.show()

In [ ]:
df = data[data['Symbol'] == 'BTC/USDT']
for i in range(10, len(price)):
    print(i)
    max_idx = list(argrelextrema(price.values[:i], np.greater, order = 20)[0])
    min_idx = list(argrelextrema(price.values[:i], np.less, order = 20)[0])
    peaks_idx = max_idx + min_idx + [len(price.values[:i]) - 1]
    peaks_idx.sort()
    
    current_idx = peaks_idx[-5:]
    current_pat = price.values[current_idx]
    plt.plot(price.values[:i])
    plt.plot(current_idx, current_pat, c='b')
    plt.show()

In [ ]:
df = data[data['Symbol'] == 'BTC/USDT'].tail(50)
dt = df.Datetime

df = df.set_index('Datetime')

price = df.Close

high = df.High

low = df.Low

max_idx = list(argrelextrema(high.values, np.greater, order = 5)[0])
min_idx = list(argrelextrema(low.values, np.less, order = 5)[0])

peak_1 = high.values[max_idx]
peak_2 = low.values[min_idx]

peaks_p = list(peak_1) + list(peak_2)

peaks_idx = list(max_idx) + list(min_idx)

peaks_idx_dt = np.array(dt.values[peaks_idx])

peaks_p = np.array(list(peak_1) + list(peak_2))

final_df = pd.DataFrame({"price":peaks_p, "datetime":peaks_idx_dt})

final_df = final_df.sort_values(by=['datetime'])

peaks_idx_dt = final_df.datetime

peaks_p = final_df.price


In [ ]:
current_idx = peaks_idx_dt[-5:]
current_idx

In [ ]:
current_idx = np.array(list(final_df.datetime[-5:]))

current_pat = np.array(list(final_df.price[-5:]))

In [ ]:
plt.plot(high, c='g')
plt.plot(low, c='r')
plt.xticks(rotation=45)
plt.scatter(dt.values[max_idx], peak_1, c='green')
plt.scatter(dt.values[min_idx], peak_2, c='red')
plt.plot(current_idx, current_pat, c='b')

plt.show()

In [ ]:
df = data[data['Symbol'] == 'BTC/USDT']
for i in range(10, len(price)):
    print(i)
    max_idx = list(argrelextrema(price.values[:i], np.greater, order = 20)[0])
    min_idx = list(argrelextrema(price.values[:i], np.less, order = 20)[0])
    peaks_idx = max_idx + min_idx + [len(price.values[:i]) - 1]
    peaks_idx.sort()
    
    current_idx = peaks_idx[-5:]
    current_pat = price.values[current_idx]
    plt.plot(price.values[:i])
    plt.plot(current_idx, current_pat, c='b')
    plt.show()

In [ ]:
df = data[data['Symbol'] == 'BTC/USDT']
df

In [ ]:
dt = df.Datetime

df = df.set_index('Datetime')

price = df.Close

high = df.High

low = df.Low

In [ ]:
for i in range(300, df.shape[0]):


    max_idx = list(argrelextrema(high.values[:i], np.greater, order = 5)[0])
    min_idx = list(argrelextrema(low.values[:i], np.less, order = 5)[0])

    peak_1 = high.values[max_idx]
    peak_2 = low.values[min_idx]

    peaks_p = list(peak_1) + list(peak_2)

    peaks_idx = list(max_idx) + list(min_idx)

    peaks_idx_dt = np.array(dt.values[peaks_idx])

    peaks_p = np.array(list(peak_1) + list(peak_2))

    final_df = pd.DataFrame({"price":peaks_p, "datetime":peaks_idx_dt})

    final_df = final_df.sort_values(by=['datetime'])

    peaks_idx_dt = final_df.datetime

    peaks_p = final_df.price
    
    current_idx = np.array(list(final_df.datetime[-5:]))

    current_pat = np.array(list(final_df.price[-5:]))
    
    plt.plot(high[:i], c='g')
    plt.plot(low[:i], c='r')
    plt.xticks(rotation=45)
    plt.scatter(dt.values[max_idx], peak_1, c='green')
    plt.scatter(dt.values[min_idx], peak_2, c='red')
    plt.plot(current_idx, current_pat, c='b')

    plt.show()

In [ ]:
plot_pat(data, "BTC/USDT", order = 10)

In [ ]:
test = detect_harmonic(data, order=10)

In [ ]:
test